In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


In [2]:
def t_diff(x, time_diff):
    for i in range(1, time_diff):
        x['t-{}_date'.format(i)] = x['date'].shift(i)
    return x    
def within_n_days(x, T, n = 1):
    ##################################################
    # Filtering accroding to w
    ##################################################
    if T == 0:
        # w = 0:使用3月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-03-01', '%Y-%m-%d')
    elif T == 1:
        # w = 1:使用2月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-02-01', '%Y-%m-%d')
    elif T == 2:
        # w = 2:使用1月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-01-01', '%Y-%m-%d')
    elif T == -1:
        # w = -1:使用4月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-04-01', '%Y-%m-%d')
    # n = 7, 14, 21, 30
    check_points = prediction_deadline - timedelta(n) # 往prediction_deadline前n天內
    x = x[ (x.date >= check_points )]
    # 如果有些使用者在預測deadline前n天內沒有資料,到時候再concat, merge的時候會是nan就把它補-1
    return x


In [44]:
T = 0
folder = 'trainW-'+ str(T)
#label
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']] 
#file1
user_logs = utils.read_multiple_csv('../../feature/{}/compressed_user_logs'.format(folder),
parse_dates = ['date']
)


100%|██████████| 40/40 [07:33<00:00, 11.35s/it]


In [46]:
s = time.time()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(user_logs)
e = time.time()
e-s

reduce memory


In [47]:
user_logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219719873 entries, 0 to 5492996
Data columns (total 9 columns):
msno          object
date          datetime64[ns]
num_25        int16
num_50        int16
num_75        int16
num_985       int16
num_100       int16
num_unq       int16
total_secs    float32
dtypes: datetime64[ns](1), float32(1), int16(6), object(1)
memory usage: 8.2+ GB


In [50]:
user_logs = user_logs.head(n = 10000)
user_logs.info()
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 9 columns):
msno          10000 non-null object
date          10000 non-null datetime64[ns]
num_25        10000 non-null int16
num_50        10000 non-null int16
num_75        10000 non-null int16
num_985       10000 non-null int16
num_100       10000 non-null int16
num_unq       10000 non-null int16
total_secs    10000 non-null float32
dtypes: datetime64[ns](1), float32(1), int16(6), object(1)
memory usage: 390.6+ KB


27

In [85]:
user_logs_copy = user_logs

In [75]:
def listening_longevity(x):
    # input_col = ['msno','date']
    x['listening_longevity'] = (x.iloc[-1].date - x.iloc[0].date).days
    return x

In [76]:
tbl = user_logs_copy.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in']
user_logs_copy = user_logs_copy.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)
user_logs_copy
tbl.reset_index(inplace = True)
tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')
tbl['log_in_ratio'] = tbl.num_log_in/tbl.listening_longevity

In [79]:
output_col = ['msno','num_log_in','listening_longevity','log_in_ratio']

In [81]:
tbl[output_col].head()

,msno,num_log_in,listening_longevity,log_in_ratio
0,+AVJ+a6tK0WiCcnSreosPc7WxB4wr6LxmTSvOzJV3No=,15,741,0.020243
1,+IkyTOYYaIHhYLvj0vw3LM9odj7OU9pPedQNQsnmUpA=,2,126,0.015873
2,+JFN9kCoK8Dl3SiwaH6gk0zM8oQlKRusDEfYn6eUI5c=,13,745,0.017450
3,+XmTHRM25FSP6OGUKjkaaw5J/wvwPo4NtagyAmQGZzU=,9,760,0.011842
4,+XrYhI4cbAZWHsUnvZ9LDfeMI1Y7NrNzhxfoJ+b68Rc=,5,558,0.008961


# within_n_days

In [ ]:
def within_n_days(x, T, n = 1):
    ##################################################
    # Filtering accroding to w
    ##################################################
    if T == 0:
        # w = 0:使用3月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-03-01', '%Y-%m-%d')
    elif T == 1:
        # w = 1:使用2月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-02-01', '%Y-%m-%d')
    elif T == 2:
        # w = 2:使用1月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-01-01', '%Y-%m-%d')
    elif T == -1:
        # w = -1:使用4月之前的資料當作history
        prediction_deadline = datetime.strptime('2017-04-01', '%Y-%m-%d')
    # n = 7, 14, 21, 30
    check_points = prediction_deadline - timedelta(n) # 往prediction_deadline前n天內
    x = x[ (x.date >= check_points )]
    # 如果有些使用者在預測deadline前n天內沒有資料,到時候再concat, merge的時候會是nan就把它補-1
    return x

In [86]:
##################################################
# n = 7
##################################################
df_ = user_logs_copy.groupby('msno').apply(within_n_days,T, n = 7).reset_index(drop = True)
#core
# tbl = df_.groupby('msno').date_diff.mean().to_frame()
# tbl.columns = ['date_diff_during_t_7-mean']
# tbl['date_diff_during_t_7-min'] = df_.groupby('msno').date_diff.min()
# tbl['date_diff_during_t_7-max'] = df_.groupby('msno').date_diff.max()
# tbl['date_diff_during_t_7-median'] = df_.groupby('msno').date_diff.median()
# tbl['date_diff_during_t_7-std'] = df_.groupby('msno').date_diff.std()
# tbl.reset_index(inplace = True)

In [87]:
df_

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,+XmTHRM25FSP6OGUKjkaaw5J/wvwPo4NtagyAmQGZzU=,2017-02-25,6,0,1,2,3,10,1597.173950
1,+uZA0+5zTJ0TUYqEPMIG3SAaLKcssPenwVRKsb+WEKs=,2017-02-26,33,0,2,1,49,78,12423.449219
2,/osNsLxx3k7if1esxb2GpYHNkHnmNPOU3292bANjBGc=,2017-02-26,1,0,2,0,5,8,1592.750977
3,/vBRWNbwbZyCRJWN9Pqilnld7iHEiF8HmyogtyBvVVE=,2017-02-22,0,1,0,0,8,9,2065.947021
4,/zHEmYjMUlUCyfdEpXjXOBWGiLxWdYJgI5p+2AYUmnw=,2017-02-27,2,4,1,0,20,6,5670.296875
5,0GxyAsgFfz5/cBtuEklIL4R3VbwNwRw55RTBmWMZpD4=,2017-02-22,11,2,1,3,26,38,7151.819824
6,3fTcBDLJAPcSoFWDE+pcTfsSR1U7wB0Jw+B0nmKpC/o=,2017-02-22,1,1,0,0,9,11,2342.511963
7,3qPOT663vrv+zh4YlN0NSQAGvnjogIUszFGCgy4VcDU=,2017-02-25,6,3,2,1,94,14,20661.630859
8,4+FN+lfw/zCYtQT+QLJTUFSEgEXFUNlSBJY1ayjlPrY=,2017-02-27,0,0,0,0,2,2,470.000000
9,56lok1BDFg1rOnnt3tPom7mg3dABV89aA8Q/+WTIlVA=,2017-02-28,0,0,0,0,6,6,1308.000000


In [88]:
user_logs_copy

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-01-05,3,3,0,0,68,36,17364.955078
1,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-03-06,1,0,1,1,97,27,24667.316406
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-05-01,3,0,0,0,38,38,9649.029297
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-07-02,4,0,1,1,33,10,10021.519531
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-08-30,3,1,0,0,4,7,1119.555054
5,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-11-07,1,0,0,0,4,5,938.021973
6,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2016-01-10,2,0,1,0,11,6,3004.068115
7,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2016-03-16,9,3,4,1,67,50,18257.660156
8,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2016-05-10,5,3,2,1,67,66,16764.205078
9,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2016-08-04,1,4,5,0,36,43,11359.565430


# num_log_in_chance and num_log_in_ratio_by_chance

In [ ]:
df_ = df.groupby('msno').apply(make_order_number)
#count
cnt = df_.groupby(['msno', 'date']).size()
cnt.name = 'cnt'
cnt = cnt.reset_index()
# chance
user_onb_max = df_.groupby('msno').order_number.max().reset_index()
user_onb_max.columns = ['msno', 'onb_max']
user_is_cancel_min = df_.groupby(['msno', 'date']).order_number.min().reset_index()
user_is_cancel_min.columns = ['msno', 'date', 'onb_min']

chance = pd.merge(user_is_cancel_min, user_onb_max, on='msno', how='left')
chance['num_log_in_chance'] = chance.onb_max - chance.onb_min +1

tbl = pd.merge(cnt, chance, on= ['msno', 'date'], how='left')
tbl['num_log_in_ratio_by_chance'] = tbl.cnt / tbl.num_log_in_chance
col = ['msno', 'num_log_in_chance', 'num_log_in_ratio_by_chance']
tbl = tbl[col]

In [ ]:
# merge
df = pd.merge(train,user_logs, on = 'msno', how = 'left')   

del user_logs
gc.collect()

In [ ]:
#

In [89]:

# merge
df = pd.merge(train,user_logs, on = 'msno', how = 'left')   




In [91]:
df = df.dropna()

In [92]:
df

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
132,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2015-04-05,9.0,8.0,8.0,1.0,9.0,29.0,4664.320801
133,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2015-07-18,10.0,1.0,0.0,0.0,24.0,20.0,6464.363770
134,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2015-11-21,6.0,2.0,0.0,1.0,15.0,13.0,4293.396973
135,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2016-03-29,2.0,0.0,1.0,0.0,2.0,4.0,628.291016
136,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2016-12-17,0.0,1.0,0.0,0.0,12.0,4.0,3892.040039
1191,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-02-08,1.0,0.0,0.0,0.0,1.0,2.0,283.459015
1192,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-04-08,7.0,0.0,1.0,1.0,9.0,11.0,2909.780029
1193,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-05-29,3.0,2.0,0.0,4.0,13.0,19.0,4105.076172
1194,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-07-21,11.0,0.0,3.0,2.0,15.0,18.0,4689.214844
1195,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-09-13,2.0,1.0,0.0,3.0,18.0,24.0,5354.514160


In [93]:
##################################################
# All history
##################################################
# count
tbl = df.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
gc.collect()   
#log_in_ratio
tbl['log_in_ratio'] = tbl.num_log_in/tbl.listening_longevity
tbl = tbl[output_col]
# # write
# tbl.to_csv('../feature/{}/num_log_in.csv'.format(folder), index = False)

# del tbl
# gc.collect()


In [94]:
tbl

,msno,num_log_in,listening_longevity,log_in_ratio
0,+AVJ+a6tK0WiCcnSreosPc7WxB4wr6LxmTSvOzJV3No=,15,741,0.020243
1,+IkyTOYYaIHhYLvj0vw3LM9odj7OU9pPedQNQsnmUpA=,2,126,0.015873
2,+JFN9kCoK8Dl3SiwaH6gk0zM8oQlKRusDEfYn6eUI5c=,13,745,0.017450
3,+XmTHRM25FSP6OGUKjkaaw5J/wvwPo4NtagyAmQGZzU=,9,760,0.011842
4,+XrYhI4cbAZWHsUnvZ9LDfeMI1Y7NrNzhxfoJ+b68Rc=,5,558,0.008961
5,+ZrMzz7rRBZmZ/ba8rR2OQkbZGVyFng7TZuqbvE6kF0=,13,696,0.018678
6,+a4QONTDxQXbRjRW415vL91wtLs7jji73rF7A+DLGsc=,10,706,0.014164
7,+aKwc2QIqADu07y87I0IFEyC423l/+LGDSZyo+fkFv0=,15,760,0.019737
8,+hQ3B6ye6uYYKrjKut3V2x7ybYY9eOaePV8atKuOSCA=,13,749,0.017356
9,+oUMqmi/LyeSGqP0xSGRPKdVZQqppVqfVPwbUhfDNkQ=,4,764,0.005236


In [99]:
##################################################
# n = 7
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 90).reset_index(drop = True)

In [100]:
df_

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,+AVJ+a6tK0WiCcnSreosPc7WxB4wr6LxmTSvOzJV3No=,2016-12-21,45.0,0.0,2.0,0.0,18.0,44.0,4266.875977
1,+AVJ+a6tK0WiCcnSreosPc7WxB4wr6LxmTSvOzJV3No=,2017-02-09,11.0,2.0,0.0,2.0,17.0,19.0,5849.387207
2,+JFN9kCoK8Dl3SiwaH6gk0zM8oQlKRusDEfYn6eUI5c=,2017-02-01,1.0,0.0,0.0,0.0,16.0,17.0,4052.937012
3,+XmTHRM25FSP6OGUKjkaaw5J/wvwPo4NtagyAmQGZzU=,2017-02-25,6.0,0.0,1.0,2.0,3.0,10.0,1597.173950
4,+ZrMzz7rRBZmZ/ba8rR2OQkbZGVyFng7TZuqbvE6kF0=,2017-01-08,0.0,0.0,0.0,0.0,9.0,9.0,2225.552002
5,+a4QONTDxQXbRjRW415vL91wtLs7jji73rF7A+DLGsc=,2017-01-04,3.0,0.0,2.0,1.0,2.0,8.0,1166.899048
6,+aKwc2QIqADu07y87I0IFEyC423l/+LGDSZyo+fkFv0=,2016-12-09,10.0,1.0,1.0,3.0,19.0,15.0,5569.939941
7,+aKwc2QIqADu07y87I0IFEyC423l/+LGDSZyo+fkFv0=,2017-02-06,16.0,1.0,2.0,0.0,11.0,14.0,3053.239014
8,+hQ3B6ye6uYYKrjKut3V2x7ybYY9eOaePV8atKuOSCA=,2016-12-02,43.0,3.0,3.0,2.0,25.0,65.0,7574.231934
9,+hQ3B6ye6uYYKrjKut3V2x7ybYY9eOaePV8atKuOSCA=,2017-01-30,21.0,2.0,0.0,2.0,3.0,28.0,1680.432983


In [101]:
tbl = df_.groupby('msno').date.size().to_frame()
tbl.columns = ['num_log_in']
tbl.reset_index(inplace = True)
# for computing log_in_ratio
user_logs_copy = df_.groupby('msno').apply(listening_longevity)
user_logs_copy.drop_duplicates('msno',inplace =True)

tbl = pd.merge(tbl,user_logs_copy, on = 'msno', how = 'left')

del user_logs_copy
gc.collect()   
#log_in_ratio
tbl['log_in_ratio'] = tbl.num_log_in/tbl.listening_longevity
tbl = tbl[output_col]


In [102]:
tbl

,msno,num_log_in,listening_longevity,log_in_ratio
0,+AVJ+a6tK0WiCcnSreosPc7WxB4wr6LxmTSvOzJV3No=,2,50,0.040000
1,+JFN9kCoK8Dl3SiwaH6gk0zM8oQlKRusDEfYn6eUI5c=,1,0,inf
2,+XmTHRM25FSP6OGUKjkaaw5J/wvwPo4NtagyAmQGZzU=,1,0,inf
3,+ZrMzz7rRBZmZ/ba8rR2OQkbZGVyFng7TZuqbvE6kF0=,1,0,inf
4,+a4QONTDxQXbRjRW415vL91wtLs7jji73rF7A+DLGsc=,1,0,inf
5,+aKwc2QIqADu07y87I0IFEyC423l/+LGDSZyo+fkFv0=,2,59,0.033898
6,+hQ3B6ye6uYYKrjKut3V2x7ybYY9eOaePV8atKuOSCA=,2,59,0.033898
7,+oUMqmi/LyeSGqP0xSGRPKdVZQqppVqfVPwbUhfDNkQ=,1,0,inf
8,+pte79/awjipUS+PKvXB1QHbiHJ2llCzg8TSmB7kMI4=,1,0,inf
9,+t290YGA+Sa99AXFpQRSU/uiPWmZBaiOfHu2YYzv6ME=,1,0,inf


In [103]:
df

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
132,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2015-04-05,9.0,8.0,8.0,1.0,9.0,29.0,4664.320801
133,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2015-07-18,10.0,1.0,0.0,0.0,24.0,20.0,6464.363770
134,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2015-11-21,6.0,2.0,0.0,1.0,15.0,13.0,4293.396973
135,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2016-03-29,2.0,0.0,1.0,0.0,2.0,4.0,628.291016
136,/LQ2vrNFkNhrsKQu9K22YKq65yidZzQJmq7fD8KYqXI=,2016-12-17,0.0,1.0,0.0,0.0,12.0,4.0,3892.040039
1191,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-02-08,1.0,0.0,0.0,0.0,1.0,2.0,283.459015
1192,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-04-08,7.0,0.0,1.0,1.0,9.0,11.0,2909.780029
1193,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-05-29,3.0,2.0,0.0,4.0,13.0,19.0,4105.076172
1194,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-07-21,11.0,0.0,3.0,2.0,15.0,18.0,4689.214844
1195,SQ1cNOn1+3VbjkM8J1mPvACI+F4xBrY3B907yCd26Yc=,2015-09-13,2.0,1.0,0.0,3.0,18.0,24.0,5354.514160


In [104]:
def t_diff(x, time_diff):
    for i in range(1, time_diff):
        x['t-{}_date'.format(i)] = x['date'].shift(i)
    return x    

In [111]:
#core
input_col = ['msno','date']
df = df[input_col].groupby('msno').apply(t_diff,2)
df['date_diff'] = [i.days for i in (df.date - df['t-1_date']) ] # 每一個trainW-0只做一次
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(df)
df.drop(['date','t-1_date'], axis = 1) # saving memory

df_ = df # copy
#date_diff
tbl = df_.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff-mean']
tbl['date_diff-min'] = df_.groupby('msno').date_diff.min()
tbl['date_diff-max'] = df_.groupby('msno').date_diff.max()
tbl['date_diff-median'] = df_.groupby('msno').date_diff.median()
tbl['date_diff-std'] = df_.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()

reduce memory


35

In [112]:
tbl

,msno,date_diff-mean,date_diff-min,date_diff-max,date_diff-median,date_diff-std
0,+AVJ+a6tK0WiCcnSreosPc7WxB4wr6LxmTSvOzJV3No=,49.333332,-1.0,62.0,51.0,14.415600
1,+IkyTOYYaIHhYLvj0vw3LM9odj7OU9pPedQNQsnmUpA=,62.500000,-1.0,126.0,62.5,89.802559
2,+JFN9kCoK8Dl3SiwaH6gk0zM8oQlKRusDEfYn6eUI5c=,57.230770,-1.0,77.0,62.0,19.079456
3,+XmTHRM25FSP6OGUKjkaaw5J/wvwPo4NtagyAmQGZzU=,84.333336,-1.0,226.0,70.0,61.000000
4,+XrYhI4cbAZWHsUnvZ9LDfeMI1Y7NrNzhxfoJ+b68Rc=,111.400002,-1.0,328.0,69.0,126.064667
5,+ZrMzz7rRBZmZ/ba8rR2OQkbZGVyFng7TZuqbvE6kF0=,53.461540,-1.0,63.0,58.0,16.741243
6,+a4QONTDxQXbRjRW415vL91wtLs7jji73rF7A+DLGsc=,70.500000,-1.0,114.0,74.5,31.440420
7,+aKwc2QIqADu07y87I0IFEyC423l/+LGDSZyo+fkFv0=,50.599998,-1.0,61.0,54.0,14.749334
8,+hQ3B6ye6uYYKrjKut3V2x7ybYY9eOaePV8atKuOSCA=,57.538460,-1.0,94.0,58.0,20.974649
9,+oUMqmi/LyeSGqP0xSGRPKdVZQqppVqfVPwbUhfDNkQ=,190.750000,-1.0,411.0,176.5,171.332764


In [107]:
tbl[tbl.msno.duplicated()]

,msno,date_diff-mean,date_diff-min,date_diff-max,date_diff-median,date_diff-std


In [113]:
##################################################
# n = 90
##################################################
df_ = df.groupby('msno').apply(within_n_days,T, n = 90).reset_index(drop = True)
#core
tbl = df_.groupby('msno').date_diff.mean().to_frame()
tbl.columns = ['date_diff_during_t_90-mean']
tbl['date_diff_during_t_90-min'] = df_.groupby('msno').date_diff.min()
tbl['date_diff_during_t_90-max'] = df_.groupby('msno').date_diff.max()
tbl['date_diff_during_t_90-median'] = df_.groupby('msno').date_diff.median()
tbl['date_diff_during_t_90-std'] = df_.groupby('msno').date_diff.std()
tbl.reset_index(inplace = True)
del df_
gc.collect()
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(tbl)

reduce memory


In [114]:
tbl

,msno,date_diff_during_t_90-mean,date_diff_during_t_90-min,date_diff_during_t_90-max,date_diff_during_t_90-median,date_diff_during_t_90-std
0,+AVJ+a6tK0WiCcnSreosPc7WxB4wr6LxmTSvOzJV3No=,50.0,50.0,50.0,50.0,0.000000
1,+JFN9kCoK8Dl3SiwaH6gk0zM8oQlKRusDEfYn6eUI5c=,64.0,64.0,64.0,64.0,-1.000000
2,+XmTHRM25FSP6OGUKjkaaw5J/wvwPo4NtagyAmQGZzU=,226.0,226.0,226.0,226.0,-1.000000
3,+ZrMzz7rRBZmZ/ba8rR2OQkbZGVyFng7TZuqbvE6kF0=,60.0,60.0,60.0,60.0,-1.000000
4,+a4QONTDxQXbRjRW415vL91wtLs7jji73rF7A+DLGsc=,114.0,114.0,114.0,114.0,-1.000000
5,+aKwc2QIqADu07y87I0IFEyC423l/+LGDSZyo+fkFv0=,56.5,54.0,59.0,56.5,3.535534
6,+hQ3B6ye6uYYKrjKut3V2x7ybYY9eOaePV8atKuOSCA=,62.0,59.0,65.0,62.0,4.242640
7,+oUMqmi/LyeSGqP0xSGRPKdVZQqppVqfVPwbUhfDNkQ=,211.0,211.0,211.0,211.0,-1.000000
8,+pte79/awjipUS+PKvXB1QHbiHJ2llCzg8TSmB7kMI4=,54.0,54.0,54.0,54.0,-1.000000
9,+t290YGA+Sa99AXFpQRSU/uiPWmZBaiOfHu2YYzv6ME=,63.0,63.0,63.0,63.0,-1.000000
